In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [3]:
# 입력 데이터 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

In [4]:
# 다층 퍼셉트론 모델 생성
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [5]:
# 다층 퍼셉트론 모델 컴파일
mlp_model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [6]:
# 모델의 구조 확인
mlp_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [7]:
# mlp_model 학습 : 5번 반복 학습 -> 점점 loss줄고 accuracy증가
mlp_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2659 - accuracy: 0.9247
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1178 - accuracy: 0.9650
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0801 - accuracy: 0.9756
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0596 - accuracy: 0.9819
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0461 - accuracy: 0.9858


In [8]:
# 모델 정확도 평가
mlp_model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0745 - accuracy: 0.9769


[0.07448011636734009, 0.9768999814987183]

In [9]:
# 합성곱 신경망을 이용한 손글씨 분류 모델 구현
# 입력 값의 형태를 4차원으로 변경
x_train_4d = x_train.reshape(-1, 28, 28, 1)
x_test_4d = x_test.reshape(-1, 28, 28, 1)

cnn_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

cnn_model.summary()
cnn_model.fit(x_train_4d, y_train, epochs=5)
cnn_model.evaluate(x_test_4d, y_test, verbose=2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

[0.02761126682162285, 0.9912999868392944]

In [ ]:
# ResNet을 이용한 손글씨 분류 모델 구현
'''
x_train_4d = x_train.reshape(-1, 28, 28, 1)
x_test_4d = x_test.reshape(-1, 28, 28, 1)

resized_x_train = tf.image.resize(x_train_4d, [32, 32])
resized_x_test = tf.image.resize(x_test_4d, [32, 32])

resnet_model = tf.keras.applications.ResNet50V2(
    input_shape=(32, 32, 1),
    classes = 10,
    weights=None
)

resnet_model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
resnet_model.fit(resized_x_train, y_train, epochs=5)
resnet_model.evaluate(resized_x_test, y_test, verbose=2)
'''

In [12]:
# 케라스 모델 변환
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\rnjst\AppData\Local\Temp\tmppuv2ku3y\assets


In [13]:
# 케라스 모델 저장
with open('./keras_model.tflite', 'wb') as f:
    f.write(tflite_model)